# SVM: Version 6

## SVM with PCA and SIFT (GrayScale)

In [1]:
import re
import cv2
import os
import glob
import time
import mlflow
import random
import shutil
import itertools

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import (
    make_scorer,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
)

In [2]:
TRAIN_DIR = "../data/train"
CLASS_FOLDERS = ["Class A", "Class B", "Class C", "Class D"]
ML_FLOW_DIRECTORY = "SVM_Logs"

In [3]:
mlflow.set_tracking_uri(ML_FLOW_DIRECTORY)
mlflow.set_experiment("SVM_PCA_SIFT_Grayscale")

2023/12/07 17:19:27 INFO mlflow.tracking.fluent: Experiment with name 'SVM_PCA_SIFT_Grayscale' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/jacob/Code/Monkey-Business/SVM/SVM_Logs/367070911128506141', creation_time=1701991167992, experiment_id='367070911128506141', last_update_time=1701991167992, lifecycle_stage='active', name='SVM_PCA_SIFT_Grayscale', tags={}>

In [4]:
pca_components = [40 * 40, 45 * 45, 50 * 50]
C_values = [0.1, 1, 10]
kernel_types = ["linear", "rbf"]
gamma_values = ["scale", "auto"]

In [5]:
hyperparameter_combinations = list(
    itertools.product(pca_components, C_values, kernel_types, gamma_values)
)
print(
    f"Total number of hyperparameter combinations: {len(hyperparameter_combinations)}"
)

Total number of hyperparameter combinations: 36


In [6]:
pca_models_dict = {}

In [7]:
def train_with_params(pca_components, C, kernel, gamma, train_images, train_labels):
    # Check if PCA model for pca_components already exists
    if pca_components in pca_models_dict:
        pca = pca_models_dict[pca_components]
        print(f"Using existing PCA model for {pca_components} components")
    else:
        # Apply PCA
        pca = PCA(n_components=pca_components)
        pca.fit(train_images)
        pca_models_dict[pca_components] = pca

    # Transform images with PCA
    train_images_pca = pca.transform(train_images)

    with mlflow.start_run() as run:
        start_time = time.time()

        # Initialize SVM model
        svm_model = svm.SVC(
            C=C, kernel=kernel, gamma=gamma, class_weight="balanced", verbose=True
        )

        # Perform cross-validation
        skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
        scoring = {
            "f1_weighted": make_scorer(f1_score, average="weighted"),
            "accuracy": "accuracy",
            "precision": make_scorer(precision_score, average="weighted"),
            "recall": make_scorer(recall_score, average="weighted"),
        }
        cv_results = cross_validate(
            svm_model,
            train_images_pca,
            train_labels,
            cv=skf,
            scoring=scoring,
            verbose=2,
        )

        # Log hyperparameters and metrics
        mlflow.log_params(
            {"pca_components": pca_components, "C": C, "kernel": kernel, "gamma": gamma}
        )
        mlflow.log_metrics(
            {
                "mean_f1_weighted": cv_results["test_f1_weighted"].mean(),
                "mean_accuracy": cv_results["test_accuracy"].mean(),
                "mean_precision": cv_results["test_precision"].mean(),
                "mean_recall": cv_results["test_recall"].mean(),
            }
        )
        end_time = time.time()
        duration = end_time - start_time
        param_details = f"PCA={pca_components}, C={C}, kernel={kernel}, gamma={gamma}"
        duration_details = f"Training duration for {param_details}: {duration} seconds"
        print(duration_details)
        run_id = run.info.run_id
        return run_id

In [8]:
sift = cv2.SIFT_create()

In [9]:
def extract_sift_features(image_path, sift_model):
    image = cv2.imread(image_path)
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    keypoints, descriptors = sift_model.detectAndCompute(grayscale_image, None)
    all_descriptors = descriptors.flatten()

    return all_descriptors

In [10]:
def load_images_and_features(folder, sift_model):
    all_features = []
    labels = []
    max_features_per_image = 0

    for class_folder in CLASS_FOLDERS:
        start_time = time.time()
        class_path = os.path.join(folder, class_folder)
        image_files = glob.glob(os.path.join(class_path, "*.png"))

        for img_file in image_files:
            descriptors_per_channel = extract_sift_features(img_file, sift_model)
            max_features_per_image = max(max_features_per_image, len(descriptors_per_channel))
            all_features.append(descriptors_per_channel)
            labels.append(class_folder)

        end_time = time.time()
        duration = end_time - start_time
        print(f"Processing {class_folder} took {duration} seconds")

    # Use the maximum number of features found in any image for padding
    for i in range(len(all_features)):
        padded_descriptors = np.pad(all_features[i], (0, max_features_per_image - len(all_features[i])))
        all_features[i] = padded_descriptors

    return np.array(all_features), np.array(labels)

In [11]:
train_images, train_labels = load_images_and_features(TRAIN_DIR, sift)

Processing Class A took 6.13006591796875 seconds
Processing Class B took 3.7601559162139893 seconds
Processing Class C took 2.664581775665283 seconds
Processing Class D took 3.2592051029205322 seconds


In [12]:
print(
    f"Shape of train_images: {train_images.shape}, Shape of train_labels: {train_labels.shape}"
)

Shape of train_images: (2796, 47616), Shape of train_labels: (2796,)


In [13]:
run_ids = []

for params in hyperparameter_combinations:
    pca_components, C, kernel, gamma = params
    run_id = train_with_params(
        pca_components, C, kernel, gamma, train_images, train_labels
    )
    run_ids.append(run_id)

[LibSVM]....*..*
optimization finished, #iter = 6435
obj = -0.000181, rho = 0.298809
nSV = 574, nBSV = 0
*.*
optimization finished, #iter = 1487
obj = -0.000014, rho = -0.368516
nSV = 414, nBSV = 0
.*.*
optimization finished, #iter = 2070
obj = -0.000036, rho = -0.253851
nSV = 447, nBSV = 0
.*.*
optimization finished, #iter = 2146
obj = -0.000023, rho = -0.584878
nSV = 563, nBSV = 0
..*.*
optimization finished, #iter = 2519
obj = -0.000036, rho = -0.396934
nSV = 564, nBSV = 0
.*.*
optimization finished, #iter = 1777
obj = -0.000016, rho = 0.260470
nSV = 568, nBSV = 0
Total nSV = 1521
[CV] END .................................................... total time=   3.2s
[LibSVM]....*..*
optimization finished, #iter = 6201
obj = -0.000148, rho = 0.341031
nSV = 595, nBSV = 0
.*
optimization finished, #iter = 1527
obj = -0.000015, rho = -0.281799
nSV = 414, nBSV = 0
.*.*
optimization finished, #iter = 2286
obj = -0.000035, rho = -0.270056
nSV = 458, nBSV = 0
..*.*
optimization finished, #iter = 

/Users/jacob/.pyenv/versions/anaconda3-2022.10/envs/monkey-business/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[CV] END .................................................... total time=   4.4s
[LibSVM]*.*
optimization finished, #iter = 1023
obj = -60.184990, rho = 0.979736
nSV = 982, nBSV = 686
*
optimization finished, #iter = 648
obj = -35.585636, rho = 0.995758
nSV = 638, nBSV = 431
*
optimization finished, #iter = 733
obj = -36.993547, rho = 1.018194
nSV = 675, nBSV = 461
*
optimization finished, #iter = 710
obj = -66.914269, rho = 0.971118
nSV = 674, nBSV = 527
*
optimization finished, #iter = 803
obj = -65.934172, rho = 1.009282
nSV = 749, nBSV = 567
*
optimization finished, #iter = 666
obj = -85.370008, rho = -0.220850
nSV = 667, nBSV = 666
Total nSV = 1682


/Users/jacob/.pyenv/versions/anaconda3-2022.10/envs/monkey-business/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[CV] END .................................................... total time=   4.4s
Training duration for PCA=1600, C=0.1, kernel=rbf, gamma=scale: 13.3922860622406 seconds
Using existing PCA model for 1600 components
[LibSVM].*
optimization finished, #iter = 1195
obj = -89.262075, rho = -0.877690
nSV = 1196, nBSV = 1195
.*
optimization finished, #iter = 1059
obj = -88.095862, rho = 0.687248
nSV = 1060, nBSV = 1059
.*
optimization finished, #iter = 1130
obj = -88.829332, rho = 0.874054
nSV = 1132, nBSV = 1130
*
optimization finished, #iter = 731
obj = -87.030192, rho = 0.687248
nSV = 732, nBSV = 731
*
optimization finished, #iter = 802
obj = -87.763662, rho = 0.874054
nSV = 804, nBSV = 802
*
optimization finished, #iter = 667
obj = -86.597449, rho = 0.874054
nSV = 668, nBSV = 667
Total nSV = 1864


...

/Users/jacob/.pyenv/versions/anaconda3-2022.10/envs/monkey-business/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[CV] END .................................................... total time=   9.1s
[LibSVM]..*.*
optimization finished, #iter = 3801
obj = -552.508255, rho = -0.270134
nSV = 1192, nBSV = 0
.*.*
optimization finished, #iter = 2946
obj = -427.651059, rho = -0.435071
nSV = 1055, nBSV = 0
..*.*
optimization finished, #iter = 3419
obj = -495.246184, rho = -0.345778
nSV = 1125, nBSV = 0
.*.*
optimization finished, #iter = 1698
obj = -353.697052, rho = -0.186903
nSV = 733, nBSV = 0
..*.*
optimization finished, #iter = 2682
obj = -398.704762, rho = -0.083437
nSV = 803, nBSV = 0
..*.*
optimization finished, #iter = 2018
obj = -329.321247, rho = 0.105105
nSV = 666, nBSV = 0
Total nSV = 1858


/Users/jacob/.pyenv/versions/anaconda3-2022.10/envs/monkey-business/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[CV] END .................................................... total time=   8.4s
[LibSVM]..*.*
optimization finished, #iter = 3849
obj = -553.570175, rho = -0.272575
nSV = 1196, nBSV = 0
.*.*
optimization finished, #iter = 2711
obj = -427.253267, rho = -0.438563
nSV = 1058, nBSV = 0
..*.*
optimization finished, #iter = 3629
obj = -496.099070, rho = -0.348096
nSV = 1129, nBSV = 0
.*.*
optimization finished, #iter = 1971
obj = -352.991787, rho = -0.188525
nSV = 732, nBSV = 0
..*.*
optimization finished, #iter = 2682
obj = -398.704762, rho = -0.083437
nSV = 803, nBSV = 0
..*.*
optimization finished, #iter = 2139
obj = -328.709704, rho = 0.106767
nSV = 665, nBSV = 0
Total nSV = 1861
[CV] END .................................................... total time=   8.4s
Training duration for PCA=2500, C=10, kernel=rbf, gamma=auto: 25.827940940856934 seconds


/Users/jacob/.pyenv/versions/anaconda3-2022.10/envs/monkey-business/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
run_metrics = {}
run_params = {}

for run_id in run_ids:
    client = mlflow.tracking.MlflowClient(ML_FLOW_DIRECTORY)
    run = client.get_run(run_id)
    metrics = run.data.metrics
    params = run.data.params
    run_metrics[run_id] = metrics
    run_params[run_id] = params

for run_id in run_ids:
    print(f"Run ID: {run_id}")
    print("Metrics:")
    metrics = run_metrics[run_id]
    for metric, value in metrics.items():
        print(f"\t{metric}: {value}")

    print("Parameters:")
    params = run_params[run_id]
    for param, value in params.items():
        print(f"\t{param}: {value}")

Run ID: 6b56a024499843039ab2958fc16562ee
Metrics:
	mean_f1_weighted: 0.5691632211261789
	mean_accuracy: 0.5701001430615165
	mean_recall: 0.5701001430615165
	mean_precision: 0.5698205792622205
Parameters:
	gamma: scale
	pca_components: 1600
	C: 0.1
	kernel: linear
Run ID: 545362cd2489446ba86df0b89a69b6bf
Metrics:
	mean_f1_weighted: 0.5691632211261789
	mean_accuracy: 0.5701001430615165
	mean_recall: 0.5701001430615165
	mean_precision: 0.5698205792622205
Parameters:
	gamma: auto
	pca_components: 1600
	C: 0.1
	kernel: linear
Run ID: 7d442c6c3dde45f89f9a27b7b11c61fe
Metrics:
	mean_f1_weighted: 0.4471229669057424
	mean_accuracy: 0.48855507868383413
	mean_recall: 0.48855507868383413
	mean_precision: 0.45342515136322376
Parameters:
	gamma: scale
	pca_components: 1600
	C: 0.1
	kernel: rbf
Run ID: 6fc4ecfedb4b4bef80a649d284836dce
Metrics:
	mean_f1_weighted: 0.08033054698280266
	mean_accuracy: 0.22103004291845493
	mean_recall: 0.22103004291845493
	mean_precision: 0.04913288143086076
Parameters:
	

In [15]:
best_run_id = max(run_metrics, key=lambda x: run_metrics[x]["mean_f1_weighted"])
best_run_params = run_params[best_run_id]
best_run_metrics = run_metrics[best_run_id]

print(f"\nBest Run ID: {best_run_id}")
print("Best Parameters:")
for key, value in best_run_params.items():
    print(f"\t{key}: {value}")

print("\nBest Run Metrics:")
for metric, value in best_run_metrics.items():
    print(f"\t{metric}: {value}")


Best Run ID: 0033d619bc9e4486b82f78ef6734c914
Best Parameters:
	gamma: scale
	pca_components: 1600
	C: 10
	kernel: rbf

Best Run Metrics:
	mean_f1_weighted: 0.6176906442818454
	mean_accuracy: 0.6219599427753933
	mean_recall: 0.6219599427753933
	mean_precision: 0.6266740114995996
